## Application of Image Representation
Q: Time 도메인의 sound wave 를 Image와 같은 2-D Timestep-Frequency 도메인으로 나타냈을 때 어떤 장점들이 있을까?

A: **Vocal(Sound Source)-Isolation** 이라는 Task에 대해 매우 effective하게 사용할 수 있습니다.

### Vocal Isolation 이란?
음원이 있을때 Vocal(목소리)를 제외한 나머지 음원만 남기는 Process를 말합니다. 어떻게 목소리를 제거할 수 있을까요?

**기존 방법**: 목소리에 해당되는 음역대(Frequency)를 제거한다.

**기존 방법의 단점**: 목소리와 같은 배경음의 음역대도 함께 제거된다. 언제까지나 눈대중으로 하는 것이기 때문에 완벽하지 않다.

**AI를 만들줄 아는 우리들의 방법**: 목소리에 해당되는 음역대(Frequency)를 AI가 알아서 제거하게 한다.

In [ ]:
# 수업자료 깃허브 Repository를 다운받아준다
!rm -rf ./NLP_2020
!git clone https://github.com/HanyangTechAI/NLP_2020.git
!rm -rf ./NLP_2020/.git

!rm -rf ./DeepIsolation
!git clone https://github.com/litcoderr/DeepIsolation.git
!rm -rf ./DeepIsolation/.git

# 필요한 패키지 (인생을 편하게 해주는 미리 만들어놓은 프로그램이라고 보면 된다) 를 설치시킨다
!cat ./NLP_2020/lab02_sequence_meets_computer_vision/requirements.txt
!echo '----------------------------------'
!pip install -r ./NLP_2020/lab02_sequence_meets_computer_vision/requirements.txt

!pip install youtube_dl
!pip install pydub
!apt install ffmpeg

In [ ]:
import youtube_dl

def download(link):
  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
  }
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info(link,download=False)
    if 'entries' in result:
      video = result['entries'][0]
      return 0
    else:
      video = result
      title = "{}-{}.mp3".format(video["title"],video['id'])
      ydl.download([link])
      return title
    
link = input("유튜브 링크를 입력하시오: ")
file_path = download(link)
file_path = file_path.replace('/', '_')
print('------------------------\n{} has been download'.format(file_path))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import tempfile
import os
import pydub
import scipy
import scipy.io.wavfile

def read_mp3(file_path, as_float = True):
    """
    Read an MP3 File into numpy data.
    :param file_path: String path to a file
    :param as_float: Cast data to float and normalize to [-1, 1]
    :return: Tuple(rate, data), where
        rate is an integer indicating samples/s
        data is an ndarray(n_samples, 2)[int16] if as_float = False
            otherwise ndarray(n_samples, 2)[float] in range [-1, 1]
    """

    path, ext = os.path.splitext(file_path)
    assert ext=='.mp3'
    mp3 = pydub.AudioSegment.from_mp3(file_path)
    _, path = tempfile.mkstemp()
    mp3.export(path, format="wav")
    rate, data = scipy.io.wavfile.read(path)
    os.remove(path)
    if as_float:
        data = data/(2**15)
    return rate, data

def plot(wave, sample_rate, start_sec, end_sec):
    start_index = int(float(start_second) * sample_rate)
    end_index = int(float(end_second) * sample_rate)
    
    plt.plot(wave[start_index:end_index])
    plt.ylabel('wave')
    plt.axis([None,None,-1,1])
    plt.show()

sample_rate, wave_data = read_mp3(file_path='./{}'.format(file_path))
# Mono 오디오 파일로 변환
wave_data = wave_data.sum(axis=1) / 2

print('wave shape: {}'.format(wave_data.shape))
print('sample_rate: {}'.format(sample_rate))

### DeepIsolation API를 통해 학습된 모델을 Import 받아봅시다

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("./DeepIsolation")

from DeepIsolation import API
from DeepIsolation.modules.util.stft import STFTConverter

api = API.Model(n_gpu=1)
model = api.getModel()

converter  = STFTConverter()

In [ ]:
# 모델에 Spectrogram 전체를 집어넣으면 용량이 너무 크기 때문에
# segment로 잘라서 모델에 집어넣는 function 을 만들어준다

def isolate(wave, rate):
    # 10초 단위로 끊어 model 에 집어넣도록 하자
    interval_sec = 5
    length_sec = wave.shape[0] // rate  # 전체 길이 in seconds
    
    result = []
    filter_vector = []
    for i in tqdm(range(length_sec//interval_sec - 1), desc='Isolating MR'):
        start_sec = i * interval_sec  # 시작 초
        end_sec = (i+1) * interval_sec  # 끝 초
        
        start_tick = start_sec * rate  # 시작 tick index
        end_tick = end_sec * rate  # 끝 tick index
        
        # STFT 변환을 start_tick에서 end_tick 까지 해준다
        magnitude, phase = converter.get_stft(wave[start_tick:end_tick])
        
        # magnitude 를 pytorch Tensor로 변환해준다
        # shape: [batch_size(1), 1, time, freq]
        input_tensor = torch.FloatTensor(magnitude).to('cuda').unsqueeze(0).unsqueeze(0)
        
        # 모델에 집어넣어준다
        # output: [batch_size(1), 1, time, freq]
        estimated_magnitude, filter_vec = model(input_tensor)
        
        result.append(estimated_magnitude.cpu().detach().numpy()[0, 0, :, :])
        filter_vector.append(filter_vec.cpu().detach().numpy()[0, 0, :, :])
    
    result = np.concatenate(result)
    filter_vector = np.concatenate(filter_vector)
    return result, filter_vector

In [ ]:
estimated_magnitude, filter_vec = isolate(wave_data, sample_rate)
print('estimated_magnitude shape: {}'.format(estimated_magnitude.shape))
print('filter_vector shape: {}'.format(filter_vec.shape))

In [ ]:
original_magnitude, original_phase = converter.get_stft(wave_data)
# estimated magnitude 가 clipping 됨을 가만하여 길이를 estimated 와 맞춰주자
original_magnitude = original_magnitude[:estimated_magnitude.shape[0], :]
original_phase = original_phase[:estimated_magnitude.shape[0], :]

print('original_magnitude_shape: {}'.format(original_magnitude.shape))

### Spectrogram이 어떻게 변화 했는지 알아봅시다

In [ ]:
fig = plt.figure()
fig.set_figwidth(15)

plt.subplot(1,3,1)
plt.xlabel('original_magnitude')
plt.imshow(original_magnitude[2000:4000, :])

plt.subplot(1,3,2)
plt.xlabel('estimated_magnitude')
plt.imshow(estimated_magnitude[2000:4000, :])

plt.subplot(1,3,3)
plt.xlabel('filter')
plt.imshow(filter_vec[2000:4000, :])

plt.show()

### 실제 Sound Wave 는 어떻게 변화하는지 알아봅시다

In [ ]:
instrument_isolated_spectrogram = np.stack((estimated_magnitude, original_phase))
instrument_isolated_wave = converter.stft2wav(instrument_isolated_spectrogram)

print('instrument_isolated_wave shape: {}'.format(instrument_isolated_wave.shape))

In [ ]:
# 파형은 근본적으로 덧셈과 뺄셈이 가능하다.
# vocal을 ioslation 하고 싶으면 어떻게 하면 될까?

vocal_isolated_wave = wave_data[:instrument_isolated_wave.shape[0]] - instrument_isolated_wave
# 모든 source가 포함된 wave_dat에서 instrument만 있는 파형을 빼면 된다
print('vocal_isolated_wave shape: {}'.format(vocal_isolated_wave.shape))

In [ ]:
start_tick = 300000
end_tick = 500000

fig = plt.figure()
fig.set_figwidth(30)
fig.set_figheight(15)

plt.subplot(3,1,1)
plt.ylim(-1, 1)
plt.plot(wave_data[start_tick:end_tick])
plt.xlabel('original')

plt.subplot(3,1,2)
plt.ylim(-1, 1)
plt.plot(instrument_isolated_wave[start_tick:end_tick])
plt.xlabel('instrument isolated')

plt.subplot(3,1,3)
plt.ylim(-1, 1)
plt.plot(vocal_isolated_wave[start_tick:end_tick])
plt.xlabel('vocal isolated')

plt.show()

### 실제 음악을 들어봅시다

In [ ]:
import IPython.display as ipd

def clip(wav_type, start_sec, end_sec): # enter(type of wave("o"|"i"|"v"), start second, end second)
    if wav_type == "original": # original
        temp = wave_data
    elif wav_type == "instrument": # inst
        temp = instrument_isolated_wave
    elif wav_type == "vocal": # vocal
        temp = vocal_isolated_wave
        
    temp = temp[int(start_sec*sample_rate): int(end_sec*sample_rate)]
    
    return temp

while True:
    wav_type = input("듣고 싶은 wave type을 입력하시오. (original, instrument, vocal): ")
    if wav_type == "":
        break
    
    start_sec = float(input("시작 초: "))
    end_sec = float(input("끝 초: "))
    
    clipped_wav = clip(wav_type, start_sec, end_sec)
    
    print('\n[{file_path}-{wav_type}] {start_sec}sec - {end_sec}sec\n'.format(
        file_path = file_path,
        wav_type = wav_type,
        start_sec = start_sec,
        end_sec = end_sec
    ))
    
    ipd.display(ipd.Audio(clipped_wav, rate=sample_rate))